In [ ]:
import pandas as pd

# Veriyi yükleyin
df = pd.read_csv('water (1).csv')

# CSV dosyasına kaydedin
df.to_csv('water (1).csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'water (1).csv'

In [ ]:

# Verilerdeki herhangi bir eksik değeri kontrol edin
print(df.isnull().sum())

# Verilerin ilk birkaç satırını gözlemleyin
print(df.head())

# Verilerin özet istatistiklerini inceleyin
print(df.describe())


Sta_ID     0
Date       0
Quarter    0
Lat_Deg    0
Depthm     0
Zone       0
T_degC     0
PO4uM      0
SiO3uM     0
NO2uM      0
NO3uM      0
Salnty     0
O2ml_L     0
dtype: int64
        Sta_ID        Date  Quarter  Lat_Deg  Depthm    Zone  T_degC  PO4uM  \
0  130.0 050.0  08/16/1959        3       25       0  Photic   25.38   0.36   
1  130.0 050.0  08/16/1959        3       25       1  Photic   25.38   0.36   
2  130.0 050.0  08/16/1959        3       25      10  Photic   25.35   0.41   
3  130.0 050.0  08/16/1959        3       25      20  Photic   23.50   0.42   
4  130.0 050.0  08/16/1959        3       25      30  Photic   21.45   0.43   

   SiO3uM  NO2uM  NO3uM  Salnty  O2ml_L  
0     1.0    0.0    0.9  34.150    4.72  
1     1.0    0.0    0.9  34.150    4.72  
2     1.0    0.0    0.8  34.180    4.14  
3     1.0    0.0    1.2  34.146    4.26  
4     1.0    0.0    1.5  34.117    4.57  
             Quarter        Lat_Deg         Depthm         T_degC  \
count  337792.000000  

In [ ]:
from sklearn.model_selection import train_test_split

# Bağımsız ve bağımlı değişkenlerin ayrılması
X = df.drop(columns=['Sta_ID', 'Date', 'Zone'])
y = df['Zone']

# Eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalizasyon
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Veri ve etiketleri ayırma
X = df.drop(columns=['Zone'])  # Bağımsız değişkenler
y = df['Zone']  # Bağımlı değişken

# Kategorik verileri sayısal verilere dönüştürme (One-Hot Encoding veya Label Encoding)
# Eğer daha fazla kategorik değişkeniniz varsa benzer şekilde encode edin
X = pd.get_dummies(X)

# Label Encoding for the target variable if it is categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Verileri numpy array'e dönüştürme
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Verileri normalize etme
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayırma
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelin oluşturulması
model = Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Erken Durdurma
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modelin eğitilmesi
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping], batch_size=32)
